In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION = "" #add your own credentials for subscription, resource_group, ws_name
RESOURCE_GROUP = "sponsorship_rg"
WS_NAME = "IOTprojectWorkSpace"
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

> [!NOTE]
> Creating MLClient will not connect to the workspace. The client initialization is lazy, it will wait for the first time it needs to make a call (this will happen in the next code cell).

In [2]:
# Verify that the handle works correctly.
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location, ":", ws.resource_group)

eastus : sponsorship_rg


## Create training script

Let's start by creating the training script - the *main.py* Python file.

First create a source folder for the script:

In [ ]:
!pip install -q kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/global-weather-repository")

print("Path to dataset files:", path)

## -- Update path ---, print above path (one in the above cell, append /GlobalWeatherRepository.csv at the end) ---

In [ ]:
!mv '/home/azureuser/.cache/kagglehub/datasets/nelgiriyewithana/global-weather-repository/versions/444/GlobalWeatherRepository.csv' 'src'

In [3]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

In [19]:
%%writefile {train_src_dir}/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=int, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--n_estimators", required=False, default=200, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", default="aqi_prediction_model", type=str, help="model name")
    args = parser.parse_args()
   
    # Start Logging
    mlflow.start_run()

    # enable autologging
    mlflow.sklearn.autolog()

    ###################
    #<prepare the data>
    ###################
    
   
    credit_df = pd.read_csv("GlobalWeatherRepository.csv")
    print(credit_df.columns)
    credit_df  = credit_df[['humidity','temperature_celsius','air_quality_Carbon_Monoxide','air_quality_us-epa-index' ]]
    credit_df.dropna(inplace=True)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)



    train_df, test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
    )
    ####################
    #</prepare the data>
    ####################

    ##################
    #<train the model>
    ##################
    # Extracting the label column
    y_train = train_df.pop('air_quality_us-epa-index').values

    # convert the dataframe values to array
    X_train = train_df.values

    # Extracting the label column
    y_test = test_df.pop('air_quality_us-epa-index').values

    # convert the dataframe values to array
    X_test = test_df.values

    print(f"Training with data of shape {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    ###################
    #</train the model>
    ###################

    ##########################
    #<save and register model>
    ##########################
    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.registered_model_name, "trained_model"),
    )
    ###########################
    #</save and register model>
    ###########################
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

Overwriting main.py


As you can see in this script, once the model is trained, the model file is saved and registered to the workspace. Now you can use the registered model in inferencing endpoints.

You might need to select **Refresh** to see the new folder and script in your **Files**.

![refresh](./media/refresh.png)

## Configure the command

Now that you have a script that can perform the desired tasks, and a compute cluster to run the script, you'll use a general purpose **command** that can run command line actions. This command line action can directly call system commands or run a script. 

Here, you'll create input variables to specify the input data, split ratio, learning rate and registered model name.  The command script will:
* Use an *environment* that defines software and runtime libraries needed for the training script. Azure Machine Learning provides many curated or ready-made environments, which are useful for common training and inference scenarios. You'll use one of those environments here.  In the [Train a model](train-model.ipynb) tutorial, you'll learn how to create a custom environment. 
* Configure the command line action itself - `python main.py` in this case. The inputs/outputs are accessible in the command via the `${{ ... }}` notation.
* In this sample, we access the data from a file on the internet. 
* Since a compute resource was not specified, the script will be run on a [serverless compute cluster](https://learn.microsoft.com/azure/machine-learning/how-to-use-serverless-compute?view=azureml-api-2&tabs=python) that is automatically created.


In [21]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "aqi_model101"

job = command(
    inputs=dict(
        data=123,
        test_train_ratio=0.1,
        learning_rate=0.25,
        registered_model_name=registered_model_name,
    ),
    code="./src/",  # location of source code
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    display_name="AIR QUALITY INDEX",
)

## Submit the job 

It's now time to submit the job to run in Azure Machine Learning. This time you'll use `create_or_update`  on `ml_client`.

In [22]:
ml_client.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (11.07 MBs): 100%|██████

Experiment,Name,Type,Status,Details Page
Users,dreamy_neck_30fnnby52j,command,Starting,Link to Azure Machine Learning studio


## View job output and wait for job completion

View the job in Azure Machine Learning studio by selecting the link in the output of the previous cell. 

The output of this job will look like this in the Azure Machine Learning studio. Explore the tabs for various details like metrics, outputs etc. Once completed, the job will register a model in your workspace as a result of training. 

![Screenshot that shows the job overview](./media/view-job.gif "View the job in studio")

> [!IMPORTANT]
> Wait until the status of the job is complete before returning to this notebook to continue. The job will take 2 to 3 minutes to run. It could take longer (up to 10 minutes) if the compute cluster has been scaled down to zero nodes and custom environment is still building.

## Deploy the model as an online endpoint

Now deploy your machine learning model as a web service in the Azure cloud, an [`online endpoint`](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).

To deploy a machine learning service, you'll use the model you registered.

## Create a new online endpoint

Now that you have a registered model, it's time to create your online endpoint. The endpoint name needs to be unique in the entire Azure region. For this tutorial, you'll create a unique name using [`UUID`](https://en.wikipedia.org/wiki/Universally_unique_identifier#:~:text=A%20universally%20unique%20identifier%20).

In [23]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

Create the endpoint:

In [24]:
# Expect the endpoint creation to take a few minutes
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint for iot project",
    auth_mode="key",
    
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpoint credit-endpoint-ddde9ff7 provisioning state: Succeeded


> [!NOTE]
> Expect the endpoint creation to take a few minutes.

Once the endpoint has been created, you can retrieve it as below:

In [25]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "credit-endpoint-ddde9ff7" with provisioning state "Succeeded" is retrieved


## Deploy the model to the endpoint

Once the endpoint is created, deploy the model with the entry script. Each endpoint can have multiple deployments. Direct traffic to these deployments can be specified using rules. Here you'll create a single deployment that handles 100% of the incoming traffic. We have chosen a color name for the deployment, for example, *blue*, *green*, *red* deployments, which is arbitrary.

You can check the **Models** page on Azure Machine Learning studio, to identify the latest version of your registered model. Alternatively, the code below will retrieve the latest version number for you to use.

In [26]:
# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)
print(f'Latest model is version "{latest_model_version}" ')

Latest model is version "1" 


Deploy the latest version of the model.  

In [27]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# Expect this deployment to take approximately 6 to 8 minutes.
# create an online deployment.
# if you run into an out of quota error, change the instance_type to a comparable VM that is available.
# Learn more on https://azure.microsoft.com/en-us/pricing/details/machine-learning/.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_E2s_v3",
    instance_count=2,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint credit-endpoint-ddde9ff7 exists


......................................................................................

> [!NOTE]
> Expect this deployment to take approximately 6 to 8 minutes.

When the deployment is done, you're ready to test it.

### Test with a sample query

Once the model is deployed to the endpoint, you can run inference with it.

Create a sample request file following the design expected in the run method in the score script.

In [28]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [31]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": ["humidity", "temperature_celsius", "air_quality_Carbon_Monoxide"],
    "index": [0, 1, 2],
    "data": [
            [22, 23, 24],
            [30, 40, 50],
            [1050, 1150, 1250]
        ]
  }
}

Overwriting ./deploy/sample-request.json


In [32]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

## Clean up resources

If you're not going to use the endpoint, delete it to stop using the resource.  Make sure no other deployments are using an endpoint before you delete it.


> [!NOTE]
> Expect the complete deletion to take approximately 20 minutes.

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

## Next Steps

You now have an Azure Machine Learning workspace, which contains a compute instance to use for your development environment.

Continue on to learn how to use the compute instance to run notebooks and scripts in the Azure Machine Learning cloud. 

|Tutorial  |Description  |
|---------|---------|
| [Tutorial: Upload, access and explore your data in Azure Machine Learning](https://learn.microsoft.com/azure/tutorial-explore-data)     |  Store large data in the cloud and retrieve it from notebooks and scripts |
| [Tutorial: Model development on a cloud workstation](https://learn.microsoft.com/azure/tutorial-cloud-workstation) | Start prototyping and developing machine learning models |
| [Tutorial: Train a model in Azure Machine Learning](https://learn.microsoft.com/azure/tutorial-train-model) |    Dive in to the details of training a model     |
| [Tutorial: Deploy a model as an online endpoint](https://learn.microsoft.com/azure/tutorial-deploy-model)  |   Dive in to the details of deploying a model      |
| [Tutorial: Create production machine learning pipelines](https://learn.microsoft.com/azure/tutorial-pipeline-python-sdk) | Split a complete machine learning task into a multistep workflow. |
